In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!python -m pip install --user  opencv-python-headless

In [ ]:
!python -m pip install --user  opencv-python-headless
!python -m pip install --user layoutparser # Install the base layoutparser library with
!python -m pip install --user  "layoutparser[layoutmodels]" # Install DL layout model toolkit
!python -m pip install --user "layoutparser[ocr]" # Install OCR toolkit

In [ ]:
!python -m pip install --user 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt install --yes libgl1 libgl1-mesa-glx libglib2.0-0

In [ ]:
#pip install -U 'git+https://github.com/nikhilweee/iopath'

* restart kernel here

In [ ]:
!pip install "Pillow<=9.5.0"

In [ ]:
import layoutparser as lp
import cv2
import numpy as np
import pdf2image
import os
from PIL import Image
import json

In [ ]:
PATH_OF_IMAGES="./images/"
TESSERACT_LANGUAGE="fra"
SHOW_IMAGE=True
PRINT_DETECTED_TITLE=False

In [ ]:
model = lp.models.Detectron2LayoutModel(
    config_path ="../config.yaml",
    model_path ="../model_final.pth",
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
)

In [ ]:
list_of_json=list()
for file in os.listdir(PATH_OF_IMAGES):
    if file.endswith(".jpg"):
        jpg_file=f"{PATH_OF_IMAGES}/{file}"
        img=Image.open(jpg_file)
        one_image_np = np.asarray(img)
        layout = model.detect(one_image_np)
        image_width, image_height= img.size
        list_of_results=[]
        for i,block in enumerate(layout.to_dict()["blocks"]):
            block_dict=dict()
            block_dict["id"]="result"+str(i)
            block_dict["type"]="rectanglelabels"
            block_dict["from_name"]="label"
            block_dict["to_name"]="image"
            block_dict["original_width"]=image_width
            block_dict["original_height"]=image_height
            block_dict["image_rotation"]=0
            values_dict=dict()
            values_dict["rotation"]=0
            values_dict["x"]=block["x_1"]*100/image_width
            values_dict["y"]=block["y_1"]*100/image_height
            width=(abs(block["x_1"]-block["x_2"]))*100/image_width
            height=(abs(block["y_1"]-block["y_2"]))*100/image_height
            values_dict["width"]=width
            values_dict["height"]=height
            values_dict["rectanglelabels"]=[block["type"]]
            block_dict["value"]=values_dict
            list_of_results.append(block_dict)
        dict_one_prediction=dict()
        dict_one_prediction["model_version"]="one"
        dict_one_prediction["score"]=0.5
        dict_one_prediction["result"]=list_of_results
        dict_data_prediction=dict()
        dict_data=dict()
        dict_data["image"]="s3://cthiounn2/stage_dares/"+jpg_file
        dict_data_prediction["data"]=dict_data
        dict_data_prediction["predictions"]=[dict_one_prediction]
        data_json=json.dumps(dict_data_prediction)
        list_of_json.append(data_json)
        with open(f"json/{file}".replace(".jpg",".json"),"w") as file_to_write:
            file_to_write.write(data_json)
with open("mega_json.json","w") as file_to_write:
        file_to_write.write("[")
        file_to_write.write(",".join(list_of_json))
        file_to_write.write("]")

In [ ]:
lp.draw_box(one_image_np, layout, box_width=3, show_element_type=True).show()